In [2]:
import sys, argparse, logging
import pymssql
import _mssql
import datetime
from decimal import Decimal
import pymongo
from pymongo import MongoClient
import re

from tmqr.settings import *

try:
    from tmqr.settings_local import *
except:
    pass


In [3]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]
mongo_collection = mongo_db['asset_index']

In [3]:
mongo_collection.create_index([('contr', pymongo.ASCENDING), 
                               ('mkt', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING)])

'contr_1_mkt_1_type_1'

In [4]:
mongo_collection.create_index([('instr', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING),
                               ('exp', pymongo.ASCENDING)])

'instr_1_type_1_exp_1'

In [6]:
%%timeit
mongo_collection.find_one({'tckr': 'US.F.CLM83.830520'})

1000 loops, best of 3: 837 µs per loop


In [10]:
%%timeit
mongo_collection.find_one({'tckr': 'US.P.F-GCM18-180627.180425@860.0'})

1000 loops, best of 3: 800 µs per loop


In [11]:
len('US.P.F-GCM18-180627.180425@860.0')

32

In [9]:
%%timeit
regx = re.compile("^US.F.CLM83.*", re.IGNORECASE)
mongo_collection.find_one({'contr':  "GCM18", 'mkt': 'US', 'type': 'F'})
#mongo_collection.find_one({'tickr':  regx})

The slowest run took 8.53 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 833 µs per loop


In [40]:
%%timeit
mongo_collection.find({'instr': 'US.CL', 'mkt': 'US', 'type': 'P', 'exp': {'$gt': datetime.datetime(2015, 5, 20, 0, 0)}}).count()

100 loops, best of 3: 2.96 ms per loop


In [42]:
mongo_collection.explain().find({'instr': 'US.CL', 'mkt': 'US', 'type': 'P', 'exp': {'$gt': datetime.datetime(2015, 5, 20, 0, 0)}}).count()

TypeError: 'Collection' object is not callable. If you meant to call the 'explain' method on a 'Collection' object it is failing because no such method exists.

# Getting futures chains

In [10]:
%timeit list(mongo_collection.find({'type':'F', 'instr': 'US.CL'}).sort('exp', 1))

100 loops, best of 3: 7.71 ms per loop


In [15]:
list(mongo_collection.find({'type':'F', 'instr': 'US.CL', 'exp': {'$gt': datetime.datetime(2011, 1, 1)}}, projection=['tckr']).sort('exp', 1))

[{'_id': ObjectId('58ad3f7833b18260b69d9f7a'), 'tckr': 'US.F.CL.G11.110120'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f7b'), 'tckr': 'US.F.CL.H11.110222'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f7c'), 'tckr': 'US.F.CL.J11.110322'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f7d'), 'tckr': 'US.F.CL.K11.110419'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f7e'), 'tckr': 'US.F.CL.M11.110520'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f7f'), 'tckr': 'US.F.CL.N11.110622'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f80'), 'tckr': 'US.F.CL.Q11.110720'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f81'), 'tckr': 'US.F.CL.U11.110822'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f82'), 'tckr': 'US.F.CL.V11.110921'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f83'), 'tckr': 'US.F.CL.X11.111020'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f84'), 'tckr': 'US.F.CL.Z11.111121'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f85'), 'tckr': 'US.F.CL.F12.111221'},
 {'_id': ObjectId('58ad3f7833b18260b69d9f86'), 'tckr': 'US.F.CL.